<a href="https://colab.research.google.com/github/bennsamuel/Pneumonia-Project/blob/master/augment_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Group 77 - Pneumonia/pickle/"

In [0]:
IMG_SIZE = 224
NUM_CLASSES = 3
CATEGORIES = ["NORMAL","BACTERIA", "VIRUS"]

%cd /content/

In [0]:
#load dataset --> method baseline!
import pickle

data = pickle.load(open("/content/drive/My Drive/Group 77 - Pneumonia/pickle/data.pickle", "rb"))

![alt text](https://www.w3resource.com/w3r_images/numpy-manipulation-reshape-function-image-a.png)

In [0]:
import numpy as np

X_normal = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)